In [1]:
# Checking Fake or Real reviews on Yelp through Machine Learning - Logistic Regression Model.
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import os

In [2]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('yelpHotelData.db')
x1 = connection.execute("select * FROM review")

In [3]:
x = connection.execute("select name FROM sqlite_master WHERE type='table' ")

In [4]:
cur = connection.cursor()
cur.execute("SELECT * FROM sqlite_master")
cur.fetchall()

[('table',
  'review',
  'review',
  2,
  'CREATE TABLE review(date varchar2(30),reviewID varchar2(30),reviewerID varchar2(30),reviewContent varchar2(800),rating INT,usefulCount INT,coolCount INT,funnyCount INT,flagged varchar2(3),hotelID varchar2(30),primary key (reviewID))'),
 ('index', 'sqlite_autoindex_review_1', 'review', 3, None),
 ('table',
  'sqlite_stat1',
  'sqlite_stat1',
  4,
  'CREATE TABLE sqlite_stat1(tbl,idx,stat)'),
 ('table',
  'sqlite_stat2',
  'sqlite_stat2',
  5,
  'CREATE TABLE sqlite_stat2(tbl,idx,sampleno,sample)'),
 ('table',
  'reviewer',
  'reviewer',
  9,
  'CREATE TABLE reviewer (\n    "reviewerID" VARCHAR2(30),\n    "name" VARCHAR2(30),\n    "location" VARCHAR2(30),\n    "yelpJoinDate" VARCHAR2(10),\n    "friendCount" INT,\n    "reviewCount" INT,\n    "firstCount" INT,\n    "usefulCount" INT,\n    "coolCount" INT,\n    "funnyCount" INT,\n    "complimentCount" INT,\n    "tipCount" INT,\n    "fanCount" INT\n)'),
 ('table',
  'hotel',
  'hotel',
  825824,
  '

In [5]:
!ls -l

total 5040696
-rw-r--r--  1 sanket  staff       1065 Apr 21 22:20 Corr_sp.csv
-rw-r--r--  1 sanket  staff      81533 Apr 21 22:45 LogisticReg_yelp.ipynb
-rw-r--r--  1 sanket  staff  656990013 Apr 21 22:14 ModifiedData.pkl
-rw-r--r--@ 1 sanket  staff  936252416 Apr 21 15:23 yelpHotelData.db
-rwxr-xr-x@ 1 sanket  staff  981304320 Dec  2  2014 yelpResData.db


In [6]:
# description = []
# Class = []

# data = x1.fetchmany(100000)

# for x in data:
#     description.append(x[3])
#     Class.append(x[8])
# trainData = {'Class' : Class, 'Description' : description}
# df_X = pd.DataFrame(trainData)

In [7]:
df = pd.read_sql("select * FROM review" ,connection)
df.shape

(688329, 10)

In [8]:
#Select only first 10k rows.
# df = df.head(6)
# df=df.dropna()

In [9]:
df['reviewID'].replace('', np.nan, inplace=True)
df.dropna(subset=['reviewID'], inplace=True)
df.head(100)

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,flagged,hotelID
0,6/8/2011,MyNjnxzZVTPq,IFTr6_6NI4CgCVavIL9k5g,Let me begin by saying that there are two kind...,5,18,11,28,N,tQfLGoolUMu2J0igcWcoZg
1,8/30/2011,BdD7fsPqHQL73hwENEDT-Q,c_-hF15XgNhlyy_TqzmdaA,The only place inside the Loop that you can st...,3,0,3,4,N,tQfLGoolUMu2J0igcWcoZg
2,6/26/2009,BfhqiyfC,CiwZ6S5ZizAFL5gypf8tLA,I have walked by the Tokyo Hotel countless tim...,5,12,14,23,N,tQfLGoolUMu2J0igcWcoZg
3,9/16/2010,Ol,nf3q2h-kSQoZK2jBY92FOg,"If you are considering staying here, watch thi...",1,8,2,6,N,tQfLGoolUMu2J0igcWcoZg
4,2/5/2010,i4HIAcNTjabdpG1K4F5Q2g,Sb3DJGdZ4Rq__CqxPbae-g,"This place is disgusting, absolutely horrible,...",3,11,4,9,N,tQfLGoolUMu2J0igcWcoZg
5,8/9/2010,veKKNAaSKWj8os,nR7zLyFOlzAYqmzgJ3DtXg,Disgusting!!! There is literally duct tape ho...,1,1,0,3,N,tQfLGoolUMu2J0igcWcoZg
6,8/11/2012,6c-ZiQkHXtp1n6VfiKDQ3g,747lP4p8dUD6RTkcsIaSGg,This hotel came up on Hotwire for $108 a night...,4,2,0,1,N,33Xc1Bk_gkSY5xb2doQ7Ng
7,7/9/2012,POWQ6FuUf3oe2ZkhmHvciA,Ij5t6VdwtasSkrpp9uAbKg,"Good location, really run down. I am surprised...",2,0,0,0,N,33Xc1Bk_gkSY5xb2doQ7Ng
8,6/19/2012,QBynYcLgIgtAd-YfnrrAtA,hSERzClUe57bCw3nCp4plA,Beautiful lobby. The rest is a dump. The eleva...,1,0,1,1,N,33Xc1Bk_gkSY5xb2doQ7Ng
9,9/14/2012,ELY3TK,OMm2VcGks3QL0p0n3_kPFw,Stayed here when I went to Chicago for a weddi...,3,2,0,0,N,33Xc1Bk_gkSY5xb2doQ7Ng


In [10]:
#Convert NR->N , YR->Y
df.loc[df['flagged'] == "NR", 'flagged'] = "N"
df.loc[df['flagged'] == "YR", 'flagged'] = "Y"

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 688328 entries, 0 to 688328
Data columns (total 10 columns):
date             688328 non-null object
reviewID         688328 non-null object
reviewerID       688328 non-null object
reviewContent    688328 non-null object
rating           688328 non-null int64
usefulCount      688328 non-null int64
coolCount        688328 non-null int64
funnyCount       688328 non-null int64
flagged          688328 non-null object
hotelID          688328 non-null object
dtypes: int64(4), object(6)
memory usage: 57.8+ MB


In [12]:
# Create a binary dependent variable Target
# Target = 1 for a Spam = Yes, 0 for Non Spam = No
df['Target'] = df['flagged'].map(lambda x : 1 \
                if x == 'Y' else 0)

In [13]:
# Dropping flagged column
df = df.drop('flagged', axis=1)


In [14]:
from textblob import TextBlob

In [15]:
#Add sentiment analysis columns
def reviewFeatures(row):
    review = row['reviewContent']
    analysis = TextBlob(review)
    return pd.Series([len(review), analysis.sentiment.polarity, analysis.sentiment.subjectivity]
           , index = ['reviewLength', 'sentimentPolarity', 'sentimentSubjectivity'])

df[['reviewLength', 'sentimentPolarity', 'sentimentSubjectivity']]  = df.apply(reviewFeatures, axis=1)

In [16]:
df.head()

,date,reviewID,reviewerID,reviewContent,rating,usefulCount,coolCount,funnyCount,hotelID,Target,reviewLength,sentimentPolarity,sentimentSubjectivity
0,6/8/2011,MyNjnxzZVTPq,IFTr6_6NI4CgCVavIL9k5g,Let me begin by saying that there are two kind...,5,18,11,28,tQfLGoolUMu2J0igcWcoZg,0,2249.0,0.025111,0.569762
1,8/30/2011,BdD7fsPqHQL73hwENEDT-Q,c_-hF15XgNhlyy_TqzmdaA,The only place inside the Loop that you can st...,3,0,3,4,tQfLGoolUMu2J0igcWcoZg,0,261.0,-0.046875,0.625000
2,6/26/2009,BfhqiyfC,CiwZ6S5ZizAFL5gypf8tLA,I have walked by the Tokyo Hotel countless tim...,5,12,14,23,tQfLGoolUMu2J0igcWcoZg,0,1239.0,0.172028,0.615385
3,9/16/2010,Ol,nf3q2h-kSQoZK2jBY92FOg,"If you are considering staying here, watch thi...",1,8,2,6,tQfLGoolUMu2J0igcWcoZg,0,80.0,0.250000,0.333333
4,2/5/2010,i4HIAcNTjabdpG1K4F5Q2g,Sb3DJGdZ4Rq__CqxPbae-g,"This place is disgusting, absolutely horrible,...",3,11,4,9,tQfLGoolUMu2J0igcWcoZg,0,251.0,-0.435000,0.505000


In [17]:
import pickle

pickle.dump(df, open("ModifiedData.pkl", "wb"))

In [18]:
# X_test['reviewContent'] = pickle.load( open(file_X_test, "rb"))

In [19]:
y = df['Target']
X = df.drop(['Target','date','reviewID','reviewerID','reviewContent','hotelID'], axis=1)

In [20]:
X.head()

,rating,usefulCount,coolCount,funnyCount,reviewLength,sentimentPolarity,sentimentSubjectivity
0,5,18,11,28,2249.0,0.025111,0.569762
1,3,0,3,4,261.0,-0.046875,0.625000
2,5,12,14,23,1239.0,0.172028,0.615385
3,1,8,2,6,80.0,0.250000,0.333333
4,3,11,4,9,251.0,-0.435000,0.505000


In [21]:
# Finding the number of flagged values as 1 and 0
y.value_counts(dropna = False)

0    420784
1    267544
Name: Target, dtype: int64

In [22]:
# Summary statistics of numeric variables
pd.options.display.float_format = '{:.2f}'.format
X.describe()

,rating,usefulCount,coolCount,funnyCount,reviewLength,sentimentPolarity,sentimentSubjectivity
count,688328.00,688328.00,688328.00,688328.00,688328.00,688328.00,688328.00
mean,3.70,2.12,1.51,1.25,764.91,0.21,0.55
std,1.08,3.44,2.99,3.19,615.44,0.18,0.12
min,1.00,0.00,0.00,0.00,0.00,-1.00,0.00
25%,3.00,0.00,0.00,0.00,348.00,0.11,0.48
50%,4.00,1.00,0.00,0.00,604.00,0.21,0.55
75%,4.00,3.00,2.00,1.00,998.00,0.31,0.62
max,5.00,177.00,170.00,246.00,5013.00,1.00,1.00


In [23]:
# # Partition the dataset into training and validation dataset
# # From the dataset of 1000 rows, pick up a random sample of 70%
# # of the rows and create a training dataset. 
# train = df.sample(frac=0.70,random_state=200)
# test =  df.drop(train.index)

In [24]:
# # Checking the frequency distribution of the target variable 
# pd.crosstab(train["Target"], "count")/10000

In [25]:
# ############# PREDICTIVE MODELLING - LOGISTIC REGRESSION ###############
# import statsmodels.formula.api as smf
# import statsmodels.api as sm

In [26]:
# train.columns

In [27]:
#train.dtypes

In [28]:
# Finding for correlation between variables
pd.options.display.float_format = '{:.2f}'.format
X.corr()
# # Correlation being saved to a csv file
corr = X.corr()
corr.to_csv("Corr_sp.csv", sep=',')

In [29]:
print(corr)

                       rating  usefulCount  coolCount  funnyCount  \
rating                   1.00         0.01       0.10       -0.02   
usefulCount              0.01         1.00       0.86        0.73   
coolCount                0.10         0.86       1.00        0.80   
funnyCount              -0.02         0.73       0.80        1.00   
reviewLength            -0.03         0.35       0.29        0.25   
sentimentPolarity        0.41        -0.09      -0.05       -0.10   
sentimentSubjectivity    0.11        -0.05      -0.04       -0.04   

                       reviewLength  sentimentPolarity  sentimentSubjectivity  
rating                        -0.03               0.41                   0.11  
usefulCount                    0.35              -0.09                  -0.05  
coolCount                      0.29              -0.05                  -0.04  
funnyCount                     0.25              -0.10                  -0.04  
reviewLength                   1.00            

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import recall_score

In [31]:

# x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=0)
# # All parameters not specified are set to their defaults
# logisticReg = LogisticRegression()
# logisticReg.fit(x_train, y_train)
# predictions_lr = logisticRegr.predict(x_test)
# # Use score method to get accuracy of model
# score1 = logisticRegr.score(x_test, y_test)
# print(score1)
# print(metrics.classification_report(y_test, predictions_lr))


In [32]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

In [33]:
Xcolumns  = X.columns
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X, y)
X_ros = pd.DataFrame(X_ros)
X_ros.columns = Xcolumns

In [34]:
#upsampling for Y variable 1

# sm = SMOTE(random_state=12, ratio = 1.0)
# x_train_res, y_train_res = sm.fit_sample(X, y)


In [35]:
# x_train_res

In [36]:
#Logistic Regression with Upsampling
from sklearn.linear_model import LogisticRegression


In [37]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_ros,y_ros, test_size=0.25, random_state=42)

In [38]:
# All parameters not specified are set to their defaults
logisticRegr = LogisticRegression()

In [39]:
logisticRegr.fit(x_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [40]:
predictions = logisticRegr.predict(x_test)

In [41]:
from sklearn import metrics
from sklearn.metrics import recall_score

In [42]:

# Use score method to get accuracy of model
score = logisticRegr.score(x_test, y_test)
print(score)

0.5614804745427583


In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics


In [44]:

print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.58      0.47      0.52    105400
           1       0.55      0.65      0.60    104992

   micro avg       0.56      0.56      0.56    210392
   macro avg       0.56      0.56      0.56    210392
weighted avg       0.56      0.56      0.56    210392



In [45]:
from sklearn import svm

In [46]:
model = svm.SVC(kernel='rbf', gamma=1) 

In [ ]:
model.fit(x_train, y_train)



In [ ]:
y_pred = model.predict(x_test)
score_svm = model.score(x_test, y_test)
print(score_svm)
print(metrics.classification_report(y_test, y_pred))